In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'h:\\Text Summarizer\\Text-Summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [5]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [7]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\ProgramData\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-04 18:20:23,667: INFO: config: PyTorch version 2.2.2 available.]


In [8]:
# class ModelTrainer:
#     def __init__(self, config: ModelTrainerConfig):
#         self.config = config
    
#     def train(self):
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#         tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
#         model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
#         seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
#         dataset_bbc = load_from_disk(self.config.data_path)

#         trainer_args = TrainingArguments(
#             output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
#             per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
#             weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
#             evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
#             gradient_accumulation_steps=self.config.gradient_accumulation_steps
#         )

#         trainer = Trainer(model=model_pegasus, args=trainer_args,
#                   tokenizer=tokenizer, data_collator=seq2seq_data_collator,
#                   train_dataset=dataset_bbc["train_dataset"], 
#                   eval_dataset=dataset_bbc["val_dataset"])
        
#         trainer.train() 

#         model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
#         tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # Load each split separately
        train_dataset = load_from_disk(os.path.join(self.config.data_path, 'train_dataset'))
        val_dataset = load_from_disk(os.path.join(self.config.data_path, 'val_dataset'))

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train_dataset,  # Use the loaded train_dataset
                  eval_dataset=val_dataset)  # Use the loaded val_dataset
        
        trainer.train() 

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [10]:
try:
    config = ConfigurationManager()
    model_tainer_config = config.get_model_trainer_config()
    model_tainer_config = ModelTrainer(config = model_tainer_config)
    model_tainer_config.train()
except Exception as e:
    raise e

[2024-05-04 18:20:28,001: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-04 18:20:28,005: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-04 18:20:28,006: INFO: common: created directory at: artifacts]
[2024-05-04 18:20:28,007: INFO: common: created directory at: artifacts/model_trainer]


 10%|█         | 10/100 [05:31<47:54, 31.94s/it] 

{'loss': 3.1559, 'grad_norm': 9.103104591369629, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.1}


 20%|██        | 20/100 [10:32<41:56, 31.45s/it]

{'loss': 2.9085, 'grad_norm': 5.78249454498291, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.2}


 29%|██▉       | 29/100 [14:56<35:10, 29.73s/it]

KeyboardInterrupt: 